# Visualizing the knowledge graph with `yfiles-jupyter-graphs`

This notebook is a partial copy of [local_search.ipynb](../../local_search.ipynb) that shows how to use `yfiles-jupyter-graphs` to add interactive graph visualizations of the parquet files  and how to visualize the result context of `graphrag` queries (see at the end of this notebook).

In [2]:
# Copyright (c) 2024 Microsoft Corporation.
# Licensed under the MIT License.

In [3]:
import os

import pandas as pd
import tiktoken

from graphrag.query.context_builder.entity_extraction import EntityVectorStoreKey
from graphrag.query.indexer_adapters import (
    read_indexer_covariates,
    read_indexer_entities,
    read_indexer_relationships,
    read_indexer_reports,
    read_indexer_text_units,
)
from graphrag.query.llm.oai.chat_openai import ChatOpenAI
from graphrag.query.llm.oai.embedding import OpenAIEmbedding
from graphrag.query.llm.oai.typing import OpenaiApiType
from graphrag.query.structured_search.local_search.mixed_context import (
    LocalSearchMixedContext,
)
from graphrag.query.structured_search.local_search.search import LocalSearch
from graphrag.vector_stores.lancedb import LanceDBVectorStore

### Load text units and graph data tables as context for local search

- In this test we first load indexing outputs from parquet files to dataframes, then convert these dataframes into collections of data objects aligning with the knowledge model.

### Load tables to dataframes

In [4]:
INPUT_DIR = "ragtest/output"
LANCEDB_URI = f"{INPUT_DIR}/lancedb"

COMMUNITY_REPORT_TABLE = "create_final_community_reports"
ENTITY_TABLE = "create_final_nodes"
ENTITY_EMBEDDING_TABLE = "create_final_entities"
RELATIONSHIP_TABLE = "create_final_relationships"
COVARIATE_TABLE = "create_final_covariates"
TEXT_UNIT_TABLE = "create_final_text_units"
COMMUNITY_LEVEL = 2

#### Read entities

In [5]:
# read nodes table to get community and degree data
entity_df = pd.read_parquet(f"{INPUT_DIR}/{ENTITY_TABLE}.parquet")
entity_embedding_df = pd.read_parquet(f"{INPUT_DIR}/{ENTITY_EMBEDDING_TABLE}.parquet")

#### Read relationships

In [6]:
relationship_df = pd.read_parquet(f"{INPUT_DIR}/{RELATIONSHIP_TABLE}.parquet")
relationships = read_indexer_relationships(relationship_df)

# Visualizing nodes and relationships with `yfiles-jupyter-graphs`

`yfiles-jupyter-graphs` is a graph visualization extension that provides interactive and customizable visualizations for structured node and relationship data.

In this case, we use it to provide an interactive visualization for the knowledge graph of the [local_search.ipynb](../../local_search.ipynb) sample by passing node and relationship lists converted from the given parquet files. The requirements for the input data is an `id` attribute for the nodes and `start`/`end` properties for the relationships that correspond to the node ids. Additional attributes can be added in the `properties` of each node/relationship dict:

In [7]:

from yfiles_jupyter_graphs import GraphWidget

# converts the entities dataframe to a list of dicts for yfiles-jupyter-graphs
def convert_entities_to_dicts(df):
    """Convert the entities dataframe to a list of dicts for yfiles-jupyter-graphs."""
    nodes_dict = {}
    for _, row in df.iterrows():
        # Create a dictionary for each row and collect unique nodes
        node_id = row["title"]
        if node_id not in nodes_dict:
            nodes_dict[node_id] = {
                "id": node_id,
                "properties": row.to_dict(),
            }
    return list(nodes_dict.values())


# converts the relationships dataframe to a list of dicts for yfiles-jupyter-graphs
def convert_relationships_to_dicts(df):
    """Convert the relationships dataframe to a list of dicts for yfiles-jupyter-graphs."""
    relationships = []
    for _, row in df.iterrows():
        # Create a dictionary for each row
        relationships.append({
            "start": row["source"],
            "end": row["target"],
            "properties": row.to_dict(),
        })
    return relationships


w = GraphWidget()
w.directed = True
w.nodes = convert_entities_to_dicts(entity_df)
w.edges = convert_relationships_to_dicts(relationship_df)

/Users/andy/works/ai/graphrag-playground2/.venv/lib/python3.11/site-packages/jupyter_client/session.py:721: UserWarning: Message serialization failed with:
Out of range float values are not JSON compliant
Supporting this message is deprecated in jupyter-client 7, please make sure your message is JSON-compliant
  content = self.pack(content)


## Configure data-driven visualization

The additional properties can be used to configure the visualization for different use cases.

In [8]:
# show title on the node
w.node_label_mapping = "title"


# map community to a color
def community_to_color(community):
    """Map a community to a color."""
    colors = [
        "crimson",
        "darkorange",
        "indigo",
        "cornflowerblue",
        "cyan",
        "teal",
        "green",
    ]
    return (
        colors[int(community) % len(colors)] if community is not None else "lightgray"
    )


def edge_to_source_community(edge):
    """Get the community of the source node of an edge."""
    source_node = next(
        (entry for entry in w.nodes if entry["properties"]["title"] == edge["start"]),
        None,
    )
    source_node_community = source_node["properties"]["community"]
    return source_node_community if source_node_community is not None else None


w.node_color_mapping = lambda node: community_to_color(node["properties"]["community"])
w.edge_color_mapping = lambda edge: community_to_color(edge_to_source_community(edge))
# map size data to a reasonable factor
# w.node_scale_factor_mapping = lambda node: 0.5 + node["properties"]["size"] * 1.5 / 20
# w.node_scale_factor_mapping = lambda node: 0.5 + 20 * 1.5 / 20
# use weight for edge thickness
w.edge_thickness_factor_mapping = "weight"

## Automatic layouts

The widget provides different automatic layouts that serve different purposes: `Circular`, `Hierarchic`, `Organic (interactiv or static)`, `Orthogonal`, `Radial`, `Tree`, `Geo-spatial`.

For the knowledge graph, this sample uses the `Circular` layout, though `Hierarchic` or `Organic` are also suitable choices.

In [9]:
# Use the circular layout for this visualization. For larger graphs, the default organic layout is often preferrable.
w.circular_layout()

## Display the graph

In [10]:
display(w)

GraphWidget(layout=Layout(height='800px', width='100%'))

# Visualizing the result context of `graphrag` queries

The result context of `graphrag` queries allow to inspect the context graph of the request. This data can similarly be visualized as graph with `yfiles-jupyter-graphs`.

## Making the request

The following cell recreates the sample queries from [local_search.ipynb](../../local_search.ipynb).

In [11]:
# setup (see also ../../local_search.ipynb)
entities = read_indexer_entities(entity_df, entity_embedding_df, COMMUNITY_LEVEL)

description_embedding_store = LanceDBVectorStore(
    collection_name="default-entity-description",
)
description_embedding_store.connect(db_uri=LANCEDB_URI)
covariate_df = pd.read_parquet(f"{INPUT_DIR}/{COVARIATE_TABLE}.parquet")
claims = read_indexer_covariates(covariate_df)
covariates = {"claims": claims}
report_df = pd.read_parquet(f"{INPUT_DIR}/{COMMUNITY_REPORT_TABLE}.parquet")
reports = read_indexer_reports(report_df, entity_df, COMMUNITY_LEVEL)
text_unit_df = pd.read_parquet(f"{INPUT_DIR}/{TEXT_UNIT_TABLE}.parquet")
text_units = read_indexer_text_units(text_unit_df)


from dotenv import load_dotenv

load_dotenv("../../../.env")

api_key = os.environ["GRAPHRAG_API_KEY"]
llm_model = os.environ["GRAPHRAG_LLM_MODEL"]
api_version = os.environ["GRAPHRAG_API_VERSION"]
base_url = os.environ["GRAPHRAG_BASE_URL"]
embedding_model = os.environ["GRAPHRAG_EMBEDDING_MODEL"]
llm = ChatOpenAI(
    api_key=api_key,
    model=llm_model,
    api_version=api_version,
    api_base=base_url,
    api_type=OpenaiApiType.AzureOpenAI,  # OpenaiApiType.OpenAI or OpenaiApiType.AzureOpenAI
    max_retries=20,
)

token_encoder = tiktoken.get_encoding("cl100k_base")

text_embedder = OpenAIEmbedding(
    api_key=api_key,
    api_base=base_url,
    api_version=api_version,
    api_type=OpenaiApiType.AzureOpenAI,
    model=embedding_model,
    deployment_name=embedding_model,
    max_retries=20,
)
context_builder = LocalSearchMixedContext(
    community_reports=reports,
    text_units=text_units,
    entities=entities,
    relationships=relationships,
    covariates=covariates,
    entity_text_embeddings=description_embedding_store,
    embedding_vectorstore_key=EntityVectorStoreKey.ID,  # if the vectorstore uses entity title as ids, set this to EntityVectorStoreKey.TITLE
    text_embedder=text_embedder,
    token_encoder=token_encoder,
)

local_context_params = {
    "text_unit_prop": 0.5,
    "community_prop": 0.1,
    "conversation_history_max_turns": 5,
    "conversation_history_user_turns_only": True,
    "top_k_mapped_entities": 10,
    "top_k_relationships": 10,
    "include_entity_rank": True,
    "include_relationship_weight": True,
    "include_community_rank": False,
    "return_candidate_context": False,
    "embedding_vectorstore_key": EntityVectorStoreKey.ID,  # set this to EntityVectorStoreKey.TITLE if the vectorstore uses entity title as ids
    "max_tokens": 12_000,  # change this based on the token limit you have on your model (if you are using a model with 8k limit, a good setting could be 5000)
}

llm_params = {
    "max_tokens": 2_000,  # change this based on the token limit you have on your model (if you are using a model with 8k limit, a good setting could be 1000=1500)
    "temperature": 0.0,
}

search_engine = LocalSearch(
    llm=llm,
    context_builder=context_builder,
    token_encoder=token_encoder,
    llm_params=llm_params,
    context_builder_params=local_context_params,
    response_type="multiple paragraphs",  # free form text describing the response type and format, can be anything, e.g. prioritized list, single paragraph, multiple paragraphs, multiple-page report
)




## Run local search on sample queries

In [12]:
result = await search_engine.asearch("날개의 의미는?")
print(result.response)

## 날개의 의미

"날개"는 다양한 문맥에서 여러 가지 의미를 가질 수 있습니다. 이 단어는 물리적, 상징적, 그리고 비유적인 의미를 모두 포함할 수 있습니다. 여기서는 "날개"의 다양한 의미를 탐구하고, 특히 주어진 데이터에서 "날개"가 어떤 의미를 가지는지 살펴보겠습니다.

### 물리적 의미

물리적으로, 날개는 새나 곤충, 비행기 등의 비행을 가능하게 하는 구조를 의미합니다. 날개는 공기역학적 설계로 인해 공기 중에서 양력을 발생시켜 비행체가 떠오를 수 있게 합니다. 이와 같은 물리적 날개는 자연과 인간의 기술 모두에서 중요한 역할을 합니다.

### 상징적 의미

상징적으로, 날개는 자유, 희망, 그리고 탈출을 의미할 수 있습니다. 날개를 가진 존재는 속박에서 벗어나 자유롭게 날아다닐 수 있는 능력을 상징합니다. 예를 들어, 천사나 페가수스와 같은 신화적 존재들은 날개를 통해 신성함과 자유로움을 나타냅니다.

### 비유적 의미

비유적으로, 날개는 성장과 발전, 그리고 새로운 시작을 의미할 수 있습니다. "날개를 펴다"라는 표현은 새로운 도전이나 기회를 맞이하는 것을 의미하며, "날개를 잃다"는 좌절이나 실패를 나타낼 수 있습니다.

### 데이터에서의 의미

주어진 데이터에서 "날개"는 이상(李霜)의 작품에서 등장하는 요소로, 그의 독특한 문체와 상징성을 반영합니다. 예를 들어, 이상은 "박제가 되어 버린 천재"라는 표현을 사용하며, 날개를 통해 자신의 피로와 정신적 상태를 묘사합니다 [Data: Sources (0)]. 이는 날개가 단순한 물리적 구조를 넘어, 그의 내면 세계와 감정을 표현하는 도구로 사용되었음을 보여줍니다.

이상은 또한 날개를 통해 자신의 비범한 발육과 세상을 보는 안목을 규정하는데 사용합니다. 이는 날개가 그의 철학적 사고와 인생관을 반영하는 중요한 상징임을 나타냅니다 [Data: Sources (0)].

## 결론

날개는 물리적, 상징적, 비유적 의미를 모두 포함하는 다층적인 개념입니다. 주어진 데이터에서 날개는 특히

In [13]:
question = "주인공의 아내에 대해 설명해줘"
result = await search_engine.asearch(question)
print(result.response)

# 주인공의 아내에 대한 설명

주인공의 아내는 이야기에서 중요한 역할을 맡고 있으며, 그녀의 행동과 관계는 이야기의 전개에 큰 영향을 미칩니다. 그녀는 주인공에게 아달린을 아스피린으로 착각하게 하여 한 달 동안 복용하게 만든 인물입니다. 이로 인해 주인공은 건강에 큰 영향을 받았고, 이는 이야기의 주요 갈등 요소 중 하나입니다 [Data: Reports (6); Entities (32); Relationships (30, 100, 98)].

## 아내의 일상과 행동

아내는 주인공의 일상 생활에서 중요한 역할을 합니다. 그녀는 집안일을 관리하고, 요리를 하고, 청소를 하는 등 가정의 중심 인물로 묘사됩니다. 그녀는 아름답고 예민한 성격을 가지고 있으며, 자주 외출하고 많은 방문객을 맞이합니다. 이러한 행동은 주인공과의 관계에 긴장감을 더합니다 [Data: Reports (8); Entities (32); Relationships (31, 46, 48)].

## 아내와 다른 남자들과의 관계

아내는 두 명의 남자, 즉 낯설은 남자와 남자와 비밀스럽게 만나는 장면이 묘사됩니다. 그녀는 이 남자들과 대문에서 속삭이거나 함께 있는 모습이 목격되며, 이는 그녀의 행동에 대한 의심을 불러일으킵니다. 이러한 장면들은 아내의 불륜 가능성이나 다른 숨겨진 활동을 암시합니다 [Data: Reports (8); Entities (51, 52); Relationships (63, 66)].

## 아내의 방과 재정적 동태

아내의 방은 화려하고 그녀의 옷과 화장품으로 가득 차 있습니다. 주인공은 종종 아내의 방을 탐험하며 그녀의 소지품에 대해 호기심을 보입니다. 또한, 아내는 주인공에게 돈을 주고, 주인공은 아내에게 돈을 주는 등 재정적인 상호작용이 자주 발생합니다. 이는 가정 내에서의 의존성과 통제의 요소를 나타냅니다 [Data: Reports (8); Entities (35, 38); Relationships (34, 40, 60, 81)].

## 아내의 비정상적인 행동

## Inspecting the context data used to generate the response

In [14]:
result.context_data["entities"].head()

,id,entity,description,number of relationships,in_context
0,23,내 아내,"The narrator's wife, described as the most bea...",5,True
1,32,아내,"The entity ""아내"" refers to the narrator's wife,...",27,True
2,75,하느님,주인공이 편안하고 즐거운 세월을 자랑하고 싶어한 대상이다,1,True
3,76,이웃,주인공이 잠든 동안 불이 난 일을 겪은 사람들이다,1,True
4,35,아내의 방,"The room of the narrator's wife, described as ...",4,True


In [15]:
result.context_data["relationships"].head()

,id,source,target,description,weight,links,in_context
0,69,아내,내,"The narrator and his wife, referred to as ""아내""...",23.0,3,True
1,31,아내,나,"The narrator, referred to as ""나,"" is married t...",33.0,2,True
2,30,아내,주인공,The 주인공 and 아내 live together and share their l...,10.0,4,True
3,32,아내,아내의 방,The room belongs to the narrator's wife and is...,1.0,1,True
4,100,아내,아달린,The narrator's wife gave them 아달린. The narrato...,9.0,2,True


## Visualizing the result context as graph

In [16]:
"""
Helper function to visualize the result context with `yfiles-jupyter-graphs`.

The dataframes are converted into supported nodes and relationships lists and then passed to yfiles-jupyter-graphs.
Additionally, some values are mapped to visualization properties.
"""


def show_graph(result):
    """Visualize the result context with yfiles-jupyter-graphs."""
    from yfiles_jupyter_graphs import GraphWidget

    if (
        "entities" not in result.context_data
        or "relationships" not in result.context_data
    ):
        msg = "The passed results do not contain 'entities' or 'relationships'"
        raise ValueError(msg)

    # converts the entities dataframe to a list of dicts for yfiles-jupyter-graphs
    def convert_entities_to_dicts(df):
        """Convert the entities dataframe to a list of dicts for yfiles-jupyter-graphs."""
        nodes_dict = {}
        for _, row in df.iterrows():
            # Create a dictionary for each row and collect unique nodes
            node_id = row["entity"]
            if node_id not in nodes_dict:
                nodes_dict[node_id] = {
                    "id": node_id,
                    "properties": row.to_dict(),
                }
        return list(nodes_dict.values())

    # converts the relationships dataframe to a list of dicts for yfiles-jupyter-graphs
    def convert_relationships_to_dicts(df):
        """Convert the relationships dataframe to a list of dicts for yfiles-jupyter-graphs."""
        relationships = []
        for _, row in df.iterrows():
            # Create a dictionary for each row
            relationships.append({
                "start": row["source"],
                "end": row["target"],
                "properties": row.to_dict(),
            })
        return relationships

    w = GraphWidget()
    # use the converted data to visualize the graph
    w.nodes = convert_entities_to_dicts(result.context_data["entities"])
    w.edges = convert_relationships_to_dicts(result.context_data["relationships"])
    w.directed = True
    # show title on the node
    w.node_label_mapping = "entity"
    # use weight for edge thickness
    w.edge_thickness_factor_mapping = "weight"
    display(w)


show_graph(result)

GraphWidget(layout=Layout(height='700px', width='100%'))